- tcpflags odtranit z agg eventu
- proto -------- || ------------
- tos takisto

In [12]:
import pandas as pd
import numpy as np
from tasks import perc_diff
import uuid
import argparse
import os
from datetime import datetime

from tasks import (
    load_config,
    load_dataset_iscx,
    get_days,
    get_file_paths,
    get_stats,
    get_counts_days,
    get_counts_hours,
    get_counts_hours_sub_mean,
    check_filters,
    export_agg_data,
    load_classes
)
from filter import (
    filter_all_days_stats,
    filter_perc_count
)
from aggregation import (
    create_events,
    aggregtion
)

# add flag option for donfig json file
config = 'config-iscx2012_ET.json'

config = load_config(config)

dataset_path = f"../{config['dataset_path']}"
dataset_config = load_config(f'{dataset_path}config.json')

data = load_dataset_iscx(dataset_path=dataset_path, inputs=dataset_config['inputs'],
                         inputs_benign=dataset_config['inputs_benign'], headers=dataset_config['headers'],
                         timedelta=dataset_config['timedelta'])

classes = load_classes(dataset_path=dataset_path, path=dataset_config['classes'])

paths = get_file_paths(data=data)
print(classes.loc[classes['sid'] == 2003068]['stage'].values[0])
if check_filters(config['filters'], dataset_config['filters']):
    all_data_filtered = data.copy()
    print(len(all_data_filtered))
    for filter in config['filters']:
        if filter['type'] == 'all_days':
            group_cols = dataset_config['filters'][filter['type']]['group_cols']
            stats = get_counts_days(data=data, group_cols=group_cols)
            col = stats[paths]
            stats = get_stats(stats, col)
            stats.reset_index(inplace=True)
            filtered_stats = filter_all_days_stats(stats, filter)
            filter_list = filtered_stats['sid'].unique().tolist()
            # Filter data based on all days stats
            all_data_filtered = all_data_filtered[~all_data_filtered.set_index('sid').index.isin(filter_list)]

        elif filter['type'] == 'count_hours':
            if filter['group_cols'] not in dataset_config['filters'][filter['type']]['group_cols_options']:
                raise Exception('wrong input for group_cols option in filter options')
            group_cols = filter['group_cols']
            if filter['sub_mean'] is True:
                if filter['mean_from'] not in dataset_config['filters'][filter['type']]['from_mean_options']:
                    raise Exception('wrong input for mean_from option in filter options')
                if filter['mean_from'] == 'benign_days':
                    inputs = dataset_config['inputs_benign']
                else:
                    inputs = dataset_config['inputs']

                stats = get_counts_hours_sub_mean(data, group_cols=group_cols, inputs=inputs)
            else:
                stats = get_counts_hours(data=data, group_cols=group_cols)
            col = stats.loc[:, '0':'23']
            stats = get_stats(stats, col)
            stats.reset_index(inplace=True)
            filtered_stats = filter_perc_count(stats, filter)
            filter_list = set(list(filtered_stats[group_cols[:-1]].itertuples(index=False, name=None)))
            all_data_filtered = all_data_filtered[~all_data_filtered.set_index(group_cols[:-1]).index.isin(filter_list)]

        print(len(all_data_filtered))

    # print(all_data_filtered.loc[(all_data_filtered['file_path'] == 'inputs/ET_alert_testbed-13jun.csv')])

    # pocetnost_end = all_data_filtered[["rule_name", "sid"]].value_counts()
    # # print(pocetnost_end)
    # 
    # # sort filtered data according
    # all_data_filtered = all_data_filtered.sort_values(by=['timestamp'])
    # all_filtered_events__class = create_events(filtered_data=all_data_filtered)
    # agg_events = aggregtion(all_filtered_events=all_filtered_events__class, delta=1800)
    # # print(len(agg_events))
    # export_agg_data(agg_events=agg_events)


NETWORK_DISC
43156
5549
4898


KeyboardInterrupt: 

In [10]:
filter_list

{('benign/ET_alert_testbed-11jun.csv', 2003068, '192.168.5.122'),
 ('benign/ET_alert_testbed-11jun.csv', 2011507, '192.168.1.103'),
 ('benign/ET_alert_testbed-11jun.csv', 2011507, '192.168.2.109'),
 ('benign/ET_alert_testbed-11jun.csv', 2011507, '192.168.4.118'),
 ('benign/ET_alert_testbed-11jun.csv', 2011507, '192.168.4.119'),
 ('benign/ET_alert_testbed-11jun.csv', 2011507, '192.168.4.120'),
 ('benign/ET_alert_testbed-11jun.csv', 2012059, '192.168.1.103'),
 ('benign/ET_alert_testbed-11jun.csv', 2012059, '192.168.1.105'),
 ('benign/ET_alert_testbed-11jun.csv', 2012059, '192.168.2.107'),
 ('benign/ET_alert_testbed-11jun.csv', 2012059, '192.168.2.110'),
 ('benign/ET_alert_testbed-11jun.csv', 2012059, '192.168.2.113'),
 ('benign/ET_alert_testbed-11jun.csv', 2012059, '192.168.3.114'),
 ('benign/ET_alert_testbed-11jun.csv', 2012059, '192.168.4.121'),
 ('benign/ET_alert_testbed-11jun.csv', 2012692, '192.168.2.113'),
 ('benign/ET_alert_testbed-11jun.csv', 2012692, '192.168.3.114'),
 ('benign/

In [2]:
len(agg_events)

254

In [3]:
export_agg_data(agg_events=agg_events)

[1900-06-11 00:01:56.971477, 192.168.2.111, 192.168.5.122, 1571, 22, ET SCAN Potential SSH Scan, 2001219,
 1900-06-11 00:27:43.679056, 192.168.4.120, 192.168.5.122, 3865, 22, ET SCAN Potential SSH Scan, 2001219,
 1900-06-11 00:38:29.030531, 192.168.4.121, 65.54.189.84, 51723, 1863, ET CHAT MSN status change, 2002192,
 1900-06-11 00:40:05.390013, 192.168.4.121, 65.54.189.84, 51723, 1863, ET CHAT MSN status change, 2002192,
 1900-06-11 05:18:52.077026, 192.168.5.122, 198.164.30.2, 1056, 53, ET DNS Query for .cc TLD, 2027758,
 1900-06-11 07:36:34.948809, 192.168.3.114, 72.246.30.144, 4145, 80, ET POLICY Suspicious Terse Request for .bmp, 2030384,
 1900-06-11 08:07:10.630248, 192.168.5.122, 198.164.30.2, 63274, 53, ET DNS Query for .cc TLD, 2027758,
 1900-06-11 08:37:11.550444, 192.168.5.122, 198.164.30.2, 65494, 53, ET DNS Query for .cc TLD, 2027758,
 1900-06-11 08:37:26.902888, 192.168.3.114, 72.246.30.144, 4754, 80, ET POLICY Suspicious Terse Request for .bmp, 2030384,
 1900-06-11 08:39